# Simulation example using Area Yield DGP

In [1]:
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm

from sklearn.ensemble import StackingRegressor, StackingClassifier
from sklearn.linear_model import RidgeCV
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression

from rdrobust import rdrobust

import doubleml as dml
from doubleml.utils import GlobalRegressor, GlobalClassifier
from doubleml.rdd import RDFlex
from doubleml.rdd import datasets

from matplotlib import pyplot as plt
from statsmodels.nonparametric.kernel_regression import KernelReg

import plotly.express as px

In [2]:
%run utils.py

## DGP Parameters

In [3]:
cutoff_dist = 0.45
cutoff_improvement = 0.0

params = dict(
    seed=17,
    n_obs=3,
    K=100,
    # origin
    origin_shape='ellipsis',
    origin_a=0.035,
    origin_b=0.01,
    origin_pertubation=0.2,
    # target
    target_center=[1.5, 0],
    target_a=0.6,
    target_b=0.3,
    # action
    action_shift=[1.0, 0],
    action_scale=1.02,
    #action_pertubation=None,
    action_pertubation=[0.001, 0.0006],
    action_drag_share=0.7,
    action_drag_scale=0.7,
    # running
    running_dist_measure='projected',
    running_mea_selection=5,
    # treatment
    treatment_dist=cutoff_dist,
    treatment_improvement=cutoff_improvement,
    treatment_random_share=0.001,
)

## DGP Visualization

In [4]:
data = datasets.dgp_area_yield(**params)

In [5]:
data = datasets.dgp_area_yield(**{**params, 'n_obs': 3000})

# improved_points = data['score_improvement'] >= 0.0
# data = {k:v[improved_points] for k,v in data.items()}

fig = px.scatter(x=data['score_distance'] ,y=data['Y'], color=data['D'], labels=dict(x="score_distance", y="Y"), hover_data={'score_improvement': data['score_improvement'], 'score_distance_act': data['score_distance_act'], 'score_improvement_act': data['score_improvement_act']}) 
fig.add_vline(x=params['treatment_dist'])

In [6]:
fig = px.scatter(x=data['score_distance'] ,y=data['score_improvement'], color=data['D'], labels=dict(x="score_distance", y="score_improvement"), hover_data={'score_improvement': data['score_improvement'], 'score_distance_act': data['score_distance_act'], 'score_improvement_act': data['score_improvement_act']}) 
fig.add_vline(x=params['treatment_dist'])
fig.add_hline(y=cutoff_improvement)

In [7]:
fig = px.scatter(x=data['score_improvement'] ,y=data['Y1'] - data['Y0'], color=data['D'], labels=dict(x="score_improvement", y="effect"), hover_data={'score_improvement': data['score_improvement'], 'score_distance_act': data['score_distance_act'], 'score_improvement_act': data['score_improvement_act']}) 
fig.add_vline(x=params['treatment_improvement'])

In [8]:
fig = px.scatter(x=data['score_distance'] ,y=data['Y1'] - data['Y0'], color=data['D'], labels=dict(x="score_distance", y="effect"), hover_data={'score_improvement': data['score_improvement'], 'score_distance_act': data['score_distance_act'], 'score_improvement_act': data['score_improvement_act']}) 
fig.add_vline(x=params['treatment_dist'])

## Oracle / Neighborhood Estimator
Estimation is done using bigger sample

In [9]:
n_obs = 10000
selected_params = {**params, 'n_obs': n_obs}
data = datasets.dgp_area_yield(**selected_params)

defiers = data["D"] != data["T"]
print(f"Defier percentage: {defiers.mean()}")

Defier percentage: 0.0216


In [10]:
cutoff = params['treatment_dist']
score = data['score_distance']
ite = data['Y1'] - data['Y0']

kernel_regression_range = 0.3
# remove defiers and restrict to a range around the cutoff
kernel_subset = (score >= cutoff - kernel_regression_range) & (score <= cutoff + kernel_regression_range)

Define all oracle effects

In [11]:
# fuzzy effect
kernel_subset_fuzzy = kernel_subset & ~defiers
kernel_reg_fuzzy = KernelReg(endog=ite[kernel_subset_fuzzy], exog=score[kernel_subset_fuzzy], var_type='c', reg_type='ll')

effect_fuzzy, _  = kernel_reg_fuzzy.fit(np.array([cutoff]))
effect_fuzzy_kernel = effect_fuzzy[0]
print(f"Estimated effect at cutoff (fuzzy): {effect_fuzzy_kernel}")

# intention to treat
kernel_reg_intend = KernelReg(endog=ite[kernel_subset], exog=score[kernel_subset], var_type='c', reg_type='ll')

effect_intend, _  = kernel_reg_intend.fit(np.array([cutoff]))
effect_intend_kernel = effect_intend[0]
print(f"Estimated effect at cutoff (intend to treat): {effect_intend_kernel}")

Estimated effect at cutoff (fuzzy): 0.04714747382400027
Estimated effect at cutoff (intend to treat): 0.0756365704796955


In [12]:
treatment_dist = params['treatment_dist']
X1_close_fuzzy = (score[kernel_subset_fuzzy] > treatment_dist - 0.02) & (score[kernel_subset_fuzzy] < treatment_dist + 0.02)
print(f'Neighborhood observations (fuzzy): {X1_close_fuzzy.sum()}')
effect_fuzzy_neighborhood = ite[kernel_subset_fuzzy][X1_close_fuzzy].mean()
print(f'Neighborhood estimator (fuzzy): {effect_fuzzy_neighborhood}')

X1_close = (score[kernel_subset] > treatment_dist - 0.02) & (score[kernel_subset] < treatment_dist + 0.02)
print(f'Neighborhood observations (intend to treat): {X1_close.sum()}')
effect_intend_neighborhood = ite[kernel_subset][X1_close].mean()
print(f'Neighborhood estimator (intend to treat): {effect_intend_neighborhood}')

Neighborhood observations (fuzzy): 165
Neighborhood estimator (fuzzy): 0.04921212121212122
Neighborhood observations (intend to treat): 209
Neighborhood estimator (intend to treat): 0.08100478468899522


## Learners

In [13]:
base_regressors = [
    ('lgbm_regressor', LGBMRegressor(n_estimators=200, learning_rate=0.01, verbose=-1, n_jobs=-1)),
    ('linear_regressor', LinearRegression()),
    ('global_regressor', GlobalRegressor(LinearRegression())),
]

stacking_regressor = StackingRegressor(
    estimators=base_regressors,
    final_estimator=LinearRegression(),
    n_jobs=-1
)

base_classifiers = [
    ('lgbm_classifier', LGBMClassifier(n_estimators=200, learning_rate=0.01, verbose=-1, n_jobs=-1)),
    ('logistic_classifier', LogisticRegression()),
    ('global_classifier', GlobalClassifier(LogisticRegression())),
]

stacking_classifier = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=LogisticRegression(),
    n_jobs=-1
)

In [14]:
linear_learner_dict = {
    "regressor": LinearRegression(),
    "classifier": LogisticRegression(),
}

global_linear_learner_dict = {
    "regressor": GlobalRegressor(LinearRegression()),
    "classifier": GlobalClassifier(LogisticRegression()),
}

lgbm_learner_dict = {
    "regressor": LGBMRegressor(n_estimators=200, learning_rate=0.01, verbose=-1, n_jobs=-1),
    "classifier": LGBMClassifier(n_estimators=200, learning_rate=0.01, verbose=-1, n_jobs=-1),
}


In [15]:
learner_dict = {
    "linear": linear_learner_dict,
    "global_linear": global_linear_learner_dict,
    "lgbm": lgbm_learner_dict,
}

## Simulation

### Single replication

In [16]:
def single_repetition(seed,
                      fs_specifications,
                      learner_dict,
                      n_obs=1000):
    res_list_fuzzy = []
    res_list_intend = []

    # generate data
    selected_params = {**params, 'n_obs': n_obs, 'seed': seed}
    data = datasets.dgp_area_yield(**selected_params)
    score = data["score_distance"]
    Y = data["Y"]
    X = data["X"].reshape(n_obs, -1)
    D = data["D"]

    # run basic rdrobust
    res_fuzzy = rdrobust(y=Y, x=score, fuzzy=D, covs=X, c=cutoff)
    res_list_fuzzy.append(
        {"rep": seed, "method": "rdrobust", "learner": "linear",
         "orcl": effect_fuzzy_kernel, "orcl_neigh": effect_fuzzy_neighborhood,
         "fs_specification": "interacted cutoff and score",
         "coef": res_fuzzy.coef.loc["Conventional", "Coeff"], "se": res_fuzzy.se.loc["Robust", "Std. Err."], 
         "2.5 %": res_fuzzy.ci.loc["Robust", "CI Lower"], "97.5 %": res_fuzzy.ci.loc["Robust", "CI Upper"]})

    res_intend = rdrobust(y=Y, x=score, covs=X, c=cutoff)
    res_list_intend.append(
        {"rep": seed, "method": "rdrobust", "learner": "linear",
         "orcl": effect_intend_kernel, "orcl_neigh": effect_intend_neighborhood,
         "fs_specification": "interacted cutoff and score",
         "coef": res_intend.coef.loc["Conventional", "Coeff"], "se": res_intend.se.loc["Robust", "Std. Err."], 
         "2.5 %": res_intend.ci.loc["Robust", "CI Lower"], "97.5 %": res_intend.ci.loc["Robust", "CI Upper"]})

    # RDFlex methods
    dml_data = dml.DoubleMLData.from_arrays(y=Y, d=D, x=X, s=score)

    for learner_name, learners in learner_dict.items():
        for fs_specification in fs_specifications:
            rdflex_model_fuzzy = RDFlex(dml_data,
                                        ml_g=learners["regressor"],
                                        ml_m=learners["classifier"],
                                        n_folds=5,
                                        n_rep=1,
                                        cutoff=cutoff,
                                        fuzzy=True,
                                        fs_specification=fs_specification)
            rdflex_model_fuzzy.fit(n_iterations=2)
            res_list_fuzzy.append(
                {"rep": seed, "method": "rdflex", "learner": learner_name,
                "orcl": effect_fuzzy_kernel, "orcl_neigh": effect_fuzzy_neighborhood,
                "fs_specification": fs_specification,
                "coef": rdflex_model_fuzzy.coef[0], "se": rdflex_model_fuzzy.se[2], 
                "2.5 %": rdflex_model_fuzzy.confint().loc["Robust", "2.5 %"], "97.5 %": rdflex_model_fuzzy.confint().loc["Robust", "97.5 %"]})

            rdflex_model_intend = RDFlex(dml_data,
                                        ml_g=learners["regressor"],
                                        n_folds=5,
                                        n_rep=1,
                                        cutoff=cutoff,
                                        fuzzy=False,
                                        fs_specification=fs_specification)
            rdflex_model_intend.fit(n_iterations=2)
            res_list_intend.append(
                {"rep": seed, "method": "rdflex", "learner": learner_name,
                "orcl": effect_intend_kernel, "orcl_neigh": effect_intend_neighborhood,
                "fs_specification": fs_specification,
                "coef": rdflex_model_intend.coef[0], "se": rdflex_model_intend.se[2],
                "2.5 %": rdflex_model_intend.confint().loc["Robust", "2.5 %"], "97.5 %": rdflex_model_intend.confint().loc["Robust", "97.5 %"]})
    
    return res_list_fuzzy, res_list_intend

In [17]:
n_rep = 1
fs_specifications = ["cutoff", "cutoff and score", "interacted cutoff and score"]

res_list_fuzzy = []
res_list_intend = []
for r in tqdm(range(n_rep), desc="Repetitions", unit="rep"):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        single_res_fuzzy, single_res_intend = single_repetition(
            r,
            fs_specifications=fs_specifications,
            learner_dict=learner_dict,
            n_obs=5000)
        res_list_fuzzy.extend(single_res_fuzzy)
        res_list_intend.extend(single_res_intend)

df_fuzzy = pd.DataFrame(res_list_fuzzy)
df_intend = pd.DataFrame(res_list_intend)

Repetitions: 100%|██████████| 1/1 [00:27<00:00, 27.12s/rep]


In [18]:
df_fuzzy.head(n=10)

,rep,method,learner,orcl,orcl_neigh,fs_specification,coef,se,2.5 %,97.5 %
0,0,rdrobust,linear,0.047147,0.049212,interacted cutoff and score,7.546265e-02,1.909005e-02,5.296928e-02,1.278009e-01
1,0,rdflex,linear,0.047147,0.049212,cutoff,1.118411e-01,3.523254e-02,1.025556e-01,2.406646e-01
2,0,rdflex,linear,0.047147,0.049212,cutoff and score,-2.293193e+09,1.056889e+10,-1.936943e+10,2.205987e+10
3,0,rdflex,linear,0.047147,0.049212,interacted cutoff and score,1.027280e+10,9.346401e+09,-6.680028e+09,2.995719e+10
4,0,rdflex,global_linear,0.047147,0.049212,cutoff,1.429396e-01,4.800630e-02,1.070020e-01,2.951832e-01
5,0,rdflex,global_linear,0.047147,0.049212,cutoff and score,-1.415615e+09,2.473689e+09,-1.418641e+09,8.278040e+09
6,0,rdflex,global_linear,0.047147,0.049212,interacted cutoff and score,-5.801662e+08,4.045069e+09,-1.030947e+10,5.546905e+09
7,0,rdflex,lgbm,0.047147,0.049212,cutoff,1.220509e-01,4.650688e-02,9.756154e-02,2.798652e-01
8,0,rdflex,lgbm,0.047147,0.049212,cutoff and score,1.515518e-01,5.463412e-02,1.152282e-01,3.293900e-01
9,0,rdflex,lgbm,0.047147,0.049212,interacted cutoff and score,1.333258e-01,5.192768e-02,9.180512e-02,2.953579e-01


In [19]:
df_intend.head(n=10)

,rep,method,learner,orcl,orcl_neigh,fs_specification,coef,se,2.5 %,97.5 %
0,0,rdrobust,linear,0.075637,0.081005,interacted cutoff and score,4.164455e-02,1.133642e-02,2.280252e-02,6.724047e-02
1,0,rdflex,linear,0.075637,0.081005,cutoff,4.379547e-02,1.590091e-02,4.058819e-02,1.029186e-01
2,0,rdflex,linear,0.075637,0.081005,cutoff and score,3.120648e+09,4.013536e+09,-7.908989e+09,7.823782e+09
3,0,rdflex,linear,0.075637,0.081005,interacted cutoff and score,2.429082e+09,3.812411e+09,-7.992459e+09,6.951918e+09
4,0,rdflex,global_linear,0.075637,0.081005,cutoff,4.683193e-02,1.685937e-02,4.343950e-02,1.095270e-01
5,0,rdflex,global_linear,0.075637,0.081005,cutoff and score,-4.772937e+08,1.041599e+09,-1.383713e+09,2.699282e+09
6,0,rdflex,global_linear,0.075637,0.081005,interacted cutoff and score,4.720735e+08,1.341320e+09,-2.667000e+09,2.590877e+09
7,0,rdflex,lgbm,0.075637,0.081005,cutoff,5.690127e-02,2.174240e-02,3.956391e-02,1.247925e-01
8,0,rdflex,lgbm,0.075637,0.081005,cutoff and score,4.816238e-02,1.917504e-02,3.698198e-02,1.121468e-01
9,0,rdflex,lgbm,0.075637,0.081005,interacted cutoff and score,3.702508e-02,1.687137e-02,2.915858e-02,9.529314e-02


In [20]:
n_rep = 100
fs_specifications = ["cutoff", "cutoff and score", "interacted cutoff and score"]

res_list_fuzzy = []
res_list_intend = []
for r in tqdm(range(n_rep), desc="Repetitions", unit="rep"):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            single_res_fuzzy, single_res_intend = single_repetition(
                r,
                fs_specifications=fs_specifications,
                learner_dict=learner_dict,
                n_obs=5000)
        
            res_list_fuzzy.extend(single_res_fuzzy)
            res_list_intend.extend(single_res_intend)
        
        except Exception as e:
            print(f"An error occurred during repetition {r}: {e}")

df_fuzzy = pd.DataFrame(res_list_fuzzy)
df_intend = pd.DataFrame(res_list_intend)

Repetitions:   9%|▉         | 9/100 [03:41<30:10, 19.90s/rep]

An error occurred during repetition 8: Matrix is not positive definite


Repetitions:  12%|█▏        | 12/100 [04:31<23:11, 15.82s/rep]

An error occurred during repetition 11: Matrix is not positive definite


Repetitions:  15%|█▌        | 15/100 [05:27<22:07, 15.62s/rep]

...

An error occurred during repetition 96: Matrix is not positive definite


Repetitions:  99%|█████████▉| 99/100 [29:30<00:11, 11.08s/rep]

An error occurred during repetition 98: Matrix is not positive definite


Repetitions: 100%|██████████| 100/100 [29:31<00:00, 17.71s/rep]

An error occurred during repetition 99: Matrix is not positive definite


## Evaluation

In [21]:
def evaluate_results(df):
    df["CI width"] = df["97.5 %"] - df["2.5 %"]
    df["Coverage"] = (df["2.5 %"] <= df["orcl"]) & (df["97.5 %"] >= df["orcl"])
    df["Coverage_neigh"] = (df["2.5 %"] <= df["orcl_neigh"]) & (df["97.5 %"] >= df["orcl_neigh"])

    df["centered coef"] = df["coef"] - df["orcl"]

    print(80*"=")
    print(f"Coverage:\n {df.groupby(["method", "learner", "fs_specification"])["Coverage"].mean()}")
    print(80*"=")
    print(f"Mean CI width:\n {df.groupby(["method", "learner", "fs_specification"])["CI width"].mean()}")
    print(f"Median CI width:\n {df.groupby(["method", "learner", "fs_specification"])["CI width"].median()}")
    print(80*"=")

    return df


In [22]:
_ = evaluate_results(df_fuzzy)

Coverage:
 method    learner        fs_specification           
rdflex    global_linear  cutoff                         0.200000
                         cutoff and score               0.966667
                         interacted cutoff and score    0.933333
          lgbm           cutoff                         0.250000
                         cutoff and score               0.283333
                         interacted cutoff and score    0.216667
          linear         cutoff                         0.216667
                         cutoff and score               0.966667
                         interacted cutoff and score    0.916667
rdrobust  linear         interacted cutoff and score    0.066667
Name: Coverage, dtype: float64
Mean CI width:
 method    learner        fs_specification           
rdflex    global_linear  cutoff                         2.101011e-01
                         cutoff and score               1.914793e+10
                         interacted cutoff and s

In [23]:
_ = evaluate_results(df_intend)

Coverage:
 method    learner        fs_specification           
rdflex    global_linear  cutoff                         0.666667
                         cutoff and score               0.950000
                         interacted cutoff and score    0.950000
          lgbm           cutoff                         0.750000
                         cutoff and score               0.750000
                         interacted cutoff and score    0.700000
          linear         cutoff                         0.666667
                         cutoff and score               0.966667
                         interacted cutoff and score    0.966667
rdrobust  linear         interacted cutoff and score    0.450000
Name: Coverage, dtype: float64
Mean CI width:
 method    learner        fs_specification           
rdflex    global_linear  cutoff                         5.372237e-02
                         cutoff and score               7.006159e+09
                         interacted cutoff and s